In [2]:
# 256个单元和ReLU激活函数的全连接隐藏层
# 10个隐藏单元且不带激活函数的全连接输出层
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
print(net(X))

tensor([[-0.0759,  0.0107, -0.0891, -0.2265,  0.1724, -0.1336, -0.3103,  0.2736,
          0.1038,  0.0267],
        [-0.1002,  0.0119,  0.0412, -0.2588,  0.0451,  0.0411, -0.3298,  0.3297,
          0.1116, -0.0536]], grad_fn=<AddmmBackward0>)


## 自定义块
- 将输入数据作为其前向传播函数的参数。
- 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
- 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
- 存储和访问前向传播计算所需的参数。
- 根据需要初始化模型参数。

In [3]:
class MLP(nn.Module):
    # 定义模型参数
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义前向传播
    def forward(self, x):
        x = F.relu(self.hidden(x))
        return self.out(x)

In [5]:
net = MLP()
print(net(X))

tensor([[-0.0442, -0.2290, -0.0032, -0.0274,  0.3919, -0.4117,  0.0994,  0.2953,
         -0.0011,  0.2677],
        [-0.0523, -0.2698,  0.0801, -0.1697,  0.2817, -0.2492, -0.0771,  0.2785,
         -0.1075,  0.2445]], grad_fn=<AddmmBackward0>)


# 顺序块
函数定义
- 一种将块逐个追加到列表中的函数
- 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module的子类实例。我们把它保存在
            # `Module`的成员变量_modules中。_modules是一个OrderDict。
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
print(net(X))

tensor([[-0.1303,  0.0178, -0.3992, -0.0673, -0.0826,  0.0270,  0.0585, -0.0910,
         -0.1943,  0.0495],
        [-0.1409,  0.0918, -0.2794, -0.0091, -0.1363, -0.0077,  0.1068,  0.0318,
         -0.2367, -0.0792]], grad_fn=<AddmmBackward0>)
